In [1]:
import torch
from google.cloud import storage
import matplotlib.pyplot as plt

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%%time

# Download data from Google Cloud Storage

from google.cloud import storage
import os

def download_directory(bucket_name, prefix, local_folder):
    """Download a GCS directory to local folder."""
    storage_client = storage.Client()

    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)  # Get all the files that are under the directory

    for blob in blobs:
        # print(blob)
        destination_filename = os.path.join(local_folder, blob.name.replace(prefix, ""))
        destination_directory = os.path.dirname(destination_filename)

        # Ensure local directory structure reflects GCS structure
        if not os.path.exists(destination_directory):
            os.makedirs(destination_directory)

        blob.download_to_filename(destination_filename)
        # print(f"Downloaded {blob.name} to {destination_filename}")

# Replace with your bucket name, directory (prefix) in GCS, and local path
bucket_name = 'ucl-remote-sensing'
gcs_directory_prefix = 'turkey_earthquake/tile_images/'
# local_path = '/content/turkey_earthquake/tile_images'

base_dir = '/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/'
local_path = base_dir + '/tile_images'

download_directory(bucket_name, gcs_directory_prefix, local_path)

CPU times: user 1min 18s, sys: 21.6 s, total: 1min 40s
Wall time: 10min 29s


In [3]:
# Mount files and append

# from google.colab import drive
# drive.mount('/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/')

import sys
sys.path.append('/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake')
# sys.path.append('/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/zoo')

In [4]:
import os

from os import path, makedirs, listdir
import sys
import numpy as np
np.random.seed(1)
import random
random.seed(1)

import torch
from torch import nn
from torch.backends import cudnn

from torch.autograd import Variable

import pandas as pd
from tqdm import tqdm
import timeit
import cv2

from zoo.models import Res34_Unet_Loc

from utils import *
from pathlib import Path

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

## Building Localization

In [ ]:
def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories
base_dir = '/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/'
# test_base_dir = '/content/turkey_earthquake/tile_images/'
test_base_dir = base_dir+'tile_images/'
pred_base_folder = base_dir + 'predict/pred34_loc/'
models_folder = base_dir + 'weights/'

if __name__ == '__main__':
    t0 = timeit.default_timer()

    # makedirs(pred_folder, exist_ok=True)

    #ensure there is a directory
    makedirs(pred_base_folder, exist_ok=True)
    city_list = list_subdirectories(test_base_dir)
    # city dir
    for city in city_list:
        # print("City:",city)
        # Ensure the directories exist
        test_dir = test_base_dir+city+'/'
        pred_folder = pred_base_folder+city+'/'
        Path(pred_folder).mkdir(exist_ok=True)
        os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
        # os.environ["CUDA_VISIBLE_DEVICES"] = sys.argv[1]

        # cudnn.benchmark = True

        models = []

        for seed in [0, 1, 2]:
            snap_to_load = 'res34_loc_{}_1_best'.format(seed)
            # print("Prepare to load model")
            model = Res34_Unet_Loc().cuda()
            # print("Load model")
            model = nn.DataParallel(model).cuda()
            # print("Try to run parallel")
            print("=> loading checkpoint '{}'".format(snap_to_load))
            checkpoint = torch.load(path.join(models_folder, snap_to_load), map_location='cpu')
            loaded_dict = checkpoint['state_dict']
            sd = model.state_dict()
            for k in model.state_dict():
                if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
                    sd[k] = loaded_dict[k]
            loaded_dict = sd
            model.load_state_dict(loaded_dict)
            print("loaded checkpoint '{}' (epoch {}, best_score {})"
                    .format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
            model.eval()
            models.append(model)


        with torch.no_grad():
            for f in tqdm(sorted(listdir(test_dir))):
                if '_pre_' in f:
                    fn = path.join(test_dir, f)

                    img = cv2.imread(fn, cv2.IMREAD_COLOR)
                    img = preprocess_inputs(img)

                    inp = []
                    inp.append(img)
                    inp.append(img[::-1, ...])
                    inp.append(img[:, ::-1, ...])
                    inp.append(img[::-1, ::-1, ...])
                    inp = np.asarray(inp, dtype='float')
                    inp = torch.from_numpy(inp.transpose((0, 3, 1, 2))).float()
                    inp = Variable(inp).cuda()

                    pred = []
                    for model in models:
                        msk = model(inp)
                        msk = torch.sigmoid(msk)
                        msk = msk.cpu().numpy()

                        pred.append(msk[0, ...])
                        pred.append(msk[1, :, ::-1, :])
                        pred.append(msk[2, :, :, ::-1])
                        pred.append(msk[3, :, ::-1, ::-1])

                    pred_full = np.asarray(pred).mean(axis=0)

                    msk = pred_full * 255
                    msk = msk.astype('uint8').transpose(1, 2, 0)
                    cv2.imwrite(path.join(pred_folder, '{0}.png'.format(f.replace('.png', '_part1.png'))), msk[..., 0], [cv2.IMWRITE_PNG_COMPRESSION, 9])

    elapsed = timeit.default_timer() - t0
    print('Time: {:.3f} min'.format(elapsed / 60))

=> loading checkpoint 'res34_loc_0_1_best'
loaded checkpoint 'res34_loc_0_1_best' (epoch 35, best_score 0.8721222407233239)
=> loading checkpoint 'res34_loc_1_1_best'
loaded checkpoint 'res34_loc_1_1_best' (epoch 53, best_score 0.8499452335526773)
=> loading checkpoint 'res34_loc_2_1_best'
loaded checkpoint 'res34_loc_2_1_best' (epoch 51, best_score 0.8597480839010446)


100%|██████████| 329/329 [03:55<00:00,  1.40it/s]


=> loading checkpoint 'res34_loc_0_1_best'
loaded checkpoint 'res34_loc_0_1_best' (epoch 35, best_score 0.8721222407233239)
=> loading checkpoint 'res34_loc_1_1_best'
loaded checkpoint 'res34_loc_1_1_best' (epoch 53, best_score 0.8499452335526773)
=> loading checkpoint 'res34_loc_2_1_best'
loaded checkpoint 'res34_loc_2_1_best' (epoch 51, best_score 0.8597480839010446)


100%|██████████| 68/68 [01:06<00:00,  1.02it/s]


=> loading checkpoint 'res34_loc_0_1_best'
loaded checkpoint 'res34_loc_0_1_best' (epoch 35, best_score 0.8721222407233239)
=> loading checkpoint 'res34_loc_1_1_best'
loaded checkpoint 'res34_loc_1_1_best' (epoch 53, best_score 0.8499452335526773)
=> loading checkpoint 'res34_loc_2_1_best'
loaded checkpoint 'res34_loc_2_1_best' (epoch 51, best_score 0.8597480839010446)


...

=> loading checkpoint 'res34_loc_0_1_best'
loaded checkpoint 'res34_loc_0_1_best' (epoch 35, best_score 0.8721222407233239)
=> loading checkpoint 'res34_loc_1_1_best'
loaded checkpoint 'res34_loc_1_1_best' (epoch 53, best_score 0.8499452335526773)
=> loading checkpoint 'res34_loc_2_1_best'
loaded checkpoint 'res34_loc_2_1_best' (epoch 51, best_score 0.8597480839010446)


100%|██████████| 442/442 [06:30<00:00,  1.13it/s]


=> loading checkpoint 'res34_loc_0_1_best'
loaded checkpoint 'res34_loc_0_1_best' (epoch 35, best_score 0.8721222407233239)
=> loading checkpoint 'res34_loc_1_1_best'
loaded checkpoint 'res34_loc_1_1_best' (epoch 53, best_score 0.8499452335526773)
=> loading checkpoint 'res34_loc_2_1_best'
loaded checkpoint 'res34_loc_2_1_best' (epoch 51, best_score 0.8597480839010446)


100%|██████████| 1470/1470 [19:38<00:00,  1.25it/s]

Time: 51.958 min


## Building Damage Classification

In [5]:
import os

from os import path, makedirs, listdir
import sys
import numpy as np
np.random.seed(1)
import random
random.seed(1)
import torch
from torch import nn
from torch.backends import cudnn

from torch.autograd import Variable

import pandas as pd
from tqdm import tqdm
import timeit
import cv2

from zoo.models import Res34_Unet_Double

from utils import *
from pathlib import Path

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

In [6]:
def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories

def get_post_disaster_filename(pre_filename,filename_list):
  city = pre_filename.split('_')[0]
  idx = '_'.join(pre_filename.split('_')[-2:])
  post_filename = ""
  result = [filename for filename in filename_list if city in filename and idx in filename and 'post_disaster' in filename]
  if len(result) > 0:
    post_filename = result[0]
  return post_filename

In [14]:
# # test function
# base_dir = '/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/'
# test_base_dir = base_dir+'tile_images/'
# city_list = list_subdirectories(test_base_dir)
# city = city_list[0]
# test_dir = test_base_dir+city+'/'
# test_list = [fn for fn in sorted(listdir(test_dir))]
# get_post_disaster_filename(test_list[8],test_list)

In [ ]:
seed = 1
base_dir = '/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/'
test_base_dir = base_dir+'tile_images/'
pred_base_folder = base_dir + 'predict/res34cls2_{}_tuned/'.format(seed)
models_folder = base_dir + 'weights/'

if __name__ == '__main__':
    t0 = timeit.default_timer()

    # vis_dev = sys.argv[2]

    # os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
    # os.environ["CUDA_VISIBLE_DEVICES"] = vis_dev

    #ensure there is a directory
    makedirs(pred_base_folder, exist_ok=True)
    city_list = list_subdirectories(test_base_dir)
    # city dir
    for city in city_list:
        print("City:",city)
        # Ensure the directories exist
        test_dir = test_base_dir+city+'/'
        pred_folder = pred_base_folder+city+'/'
        Path(pred_folder).mkdir(exist_ok=True)

        # cudnn.benchmark = True

        # filename list
        test_list = [fn for fn in sorted(listdir(test_dir))]

        models = []

        snap_to_load = 'res34_cls2_{}_tuned_best'.format(seed)
        model = Res34_Unet_Double().cuda()
        model = nn.DataParallel(model).cuda()
        print("=> loading checkpoint '{}'".format(snap_to_load))
        checkpoint = torch.load(path.join(models_folder, snap_to_load), map_location='cpu')
        loaded_dict = checkpoint['state_dict']
        sd = model.state_dict()
        for k in model.state_dict():
            if k in loaded_dict and sd[k].size() == loaded_dict[k].size():
                sd[k] = loaded_dict[k]
        loaded_dict = sd
        model.load_state_dict(loaded_dict)
        print("loaded checkpoint '{}' (epoch {}, best_score {})"
                .format(snap_to_load, checkpoint['epoch'], checkpoint['best_score']))
        model.eval()
        models.append(model)


        with torch.no_grad():
            for f in tqdm(test_list):
                if '_pre_' in f:
                    post_f = get_post_disaster_filename(f,test_list)
                    if(post_f == ''):
                      continue
                    else:
                      fn = path.join(test_dir, f)
                      post_fn = path.join(test_dir, post_f)

                      img = cv2.imread(fn, cv2.IMREAD_COLOR)
                      img2 = cv2.imread(post_fn, cv2.IMREAD_COLOR)

                      img = np.concatenate([img, img2], axis=2)
                      img = preprocess_inputs(img)

                      inp = []
                      inp.append(img)
                      inp.append(img[::-1, ...])
                      inp.append(img[:, ::-1, ...])
                      inp.append(img[::-1, ::-1, ...])
                      inp = np.asarray(inp, dtype='float')
                      inp = torch.from_numpy(inp.transpose((0, 3, 1, 2))).float()
                      inp = Variable(inp).cuda()

                      pred = []
                      for model in models:
                          msk = model(inp)
                          msk = torch.sigmoid(msk)
                          msk = msk.cpu().numpy()

                          pred.append(msk[0, ...])
                          pred.append(msk[1, :, ::-1, :])
                          pred.append(msk[2, :, :, ::-1])
                          pred.append(msk[3, :, ::-1, ::-1])

                      pred_full = np.asarray(pred).mean(axis=0)

                      msk = pred_full * 255
                      msk = msk.astype('uint8').transpose(1, 2, 0)
                      cv2.imwrite(path.join(pred_folder, '{0}'.format(f.replace('.tif', '_part1.png'))), msk[..., :3], [cv2.IMWRITE_PNG_COMPRESSION, 9])
                      cv2.imwrite(path.join(pred_folder, '{0}'.format(f.replace('.tif', '_part2.png'))), msk[..., 2:], [cv2.IMWRITE_PNG_COMPRESSION, 9])

    elapsed = timeit.default_timer() - t0
    print('Time: {:.3f} min'.format(elapsed / 60))

City: Adiyaman


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 219MB/s]


=> loading checkpoint 'res34_cls2_1_tuned_best'
loaded checkpoint 'res34_cls2_1_tuned_best' (epoch 1, best_score 0.7692094507748467)


 21%|██▏       | 70/329 [01:28<04:51,  1.13s/it]